# Random Walk

In [ ]:
from grblas import *
import numpy as np

In [ ]:
m = np.array([[0, 1, 0, 1, 1, 0, 0, 0],
              [0, 0, 0, 1, 0, 0, 0, 0],
              [0, 0, 0, 0, 1, 0, 1, 0],
              [0, 1, 0, 0, 1, 0, 0, 0],
              [1, 1, 0, 0, 0, 0, 0, 0],
              [0, 0, 1, 0, 0, 0, 0, 0],
              [0, 0, 1, 0, 0, 1, 0, 0],
              [0, 0, 1, 0, 0, 0, 0, 0]], dtype=bool)
# make symmetric (for now)
m = m + m.T
A = Matrix.ss.import_bitmapr(bitmap=m, values=m, dtype=int)
del A[7, 2]  # No outgoing edges from node 7
A

For walkers, we could do:
- each vertex
- each vertex repeated k times
- random subset of vertices, each repeated k times
- random vertices with replacement (let's do this one for now)

In [ ]:
max_steps = 10
num_walkers = 20

In [ ]:
# Frontier
walker_init = np.random.randint(0, A.nrows, size=num_walkers)
F = Matrix.from_values(np.arange(num_walkers, dtype=np.uint64), walker_init, 1)
F

In [ ]:
# Record the path taken by each walker
paths = Matrix.new(int, num_walkers, max_steps + 1)

In [ ]:
# Starting positions
paths[:, 0] = op.positionj(F).reduce_rowwise(op.any).new()
paths

In [ ]:
for i in range(max_steps):
    B = op.any_second(F @ A).new()
    if B.nvals == 0:
        break
    F = B.ss.selectk_rowwise('random', 1)
    paths[:, i + 1] = op.positionj(F).reduce_rowwise(op.any).new()

In [ ]:
paths